In [13]:
import sys
import time
import datetime as dt
from datetime import datetime
import urllib
# Number manipulation
import numpy as np


# Structure manipulation
import pandas as pd
from pandas import Series, DataFrame
from pandas.tools.plotting import scatter_matrix


# Graphing 
import seaborn as sb

import matplotlib.pyplot as plt
from matplotlib import rcParams


# Stats
import scipy as sp
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency


# Machine Learning & Preprocessing Package
import sklearn
import sklearn.metrics as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import LassoLarsIC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cross_validation import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf

from collections import Counter

import networkx as nx



# Parameters for graphs
%matplotlib inline
rcParams['figure.figsize'] = 10,10
sb.set_style('whitegrid')


# Mac
#x = pd.read_csv('/Users/magicsoccer10/Dropbox/twerk werk/Property Value/07-17.csv')

# Arch
#x = pd.read_csv('C:/Users/scott/Dropbox/twerk werk/Data/cre_values')

# Work
x = pd.read_csv('C:/Users/sstandring/Dropbox/twerk werk/Data/cre_values.csv')

df = x.dropna(axis = 0)
df.head()

,Property Address,Property City,Property Zip Code,Sale Date,Sale_Price,PropertyType,Cap_Rate,Bldg SF,Floor Area Ratio,Building Class,Building Condition,BuildingMaterials,Year Built,Number Of Units,Longitude,Latitude
0,700 Lincoln Village Cir,Larkspur,94939,7/25/2007,94500000,Multi-Family,4.28,293198,0.397103,B,Other,WoodFrame,1978.0,342,-122.507256,37.947473
1,1 Amberstone Ln,San Ramon,94582,5/4/2007,64300000,Multi-Family,4.00,237894,0.603458,B,Other,WoodFrame,2004.0,250,-121.906034,37.765747
2,5004 Hartnett Ave,Richmond,94804,7/6/2007,48443500,Multi-Family,5.50,408500,0.387195,B,Good,Unknown,1968.0,378,-122.324281,37.916949
3,7800 El Camino Real,Colma,94014,9/20/2007,44000000,Multi-Family,4.83,221000,2.402226,A,Other,ReinforcedConcrete,2005.0,153,-122.464466,37.684798
4,2795 San Leandro Blvd,San Leandro,94578,8/2/2007,28458000,Multi-Family,4.50,133974,0.616358,C,Other,Masonry,1974.0,186,-122.143928,37.715303


In [7]:
df['sqft'] = df['Bldg SF']
df['cap'] = df['Cap_Rate'].astype(int)
df['units'] = df['Number Of Units']
df['sale_date'] = pd.to_datetime(df['Sale Date'])
df['sale_quarter'] = df['sale_date'].dt.quarter
df['sale_quarter'] = df['sale_quarter'].astype('str')
df['log_price'] = np.log(df.Sale_Price)
df['log_sqft'] = np.log(df['Bldg SF'])
df['log_far'] = np.log(df['Floor Area Ratio'])
df['log_units'] = np.log(df.units)

num = LabelEncoder()
df['bin_city'] = num.fit_transform(df['Property City'].astype('str'))
df['bin_yrbuilt'] = num.fit_transform(df['Year Built'].astype('str'))

def remover(s):
    return int(s[-4:])

df['year'] = [remover(s) for s in df['Sale Date']]

df['sale_year'] = df['year'].astype('str')
df['sale_y_q'] = df.sale_year + '.' + df.sale_quarter
df['sale_year_qrt'] = df['sale_y_q'].astype('float')
df['sale_year_qrt'] = df['sale_year_qrt'] - 2000

df['yearbuilt'] = df['Year Built'].astype('str').str[:-3]
df['yearbuilt'] = df['yearbuilt'].astype(int)

df['Property Zip Code'] = df['Property Zip Code'].astype('str').str[:-1]
df['bin_zip'] = num.fit_transform(df['Property Zip Code'].astype('str'))
df['zipper'] = df['Property Zip Code'].astype('str')


C:\Users\sstandring\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\sstandring\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sstandring\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [8]:
mf = df.loc[df['PropertyType'] == 'Multi-Family']
mf = mf.loc[mf['sale_year'] != '2007']
mf = mf.loc[mf['sale_year'] != '2008']
mf = mf.loc[mf['sale_year'] != '2009']
mf = mf.loc[mf['sale_year'] != '2010']
mf = mf.loc[mf['sale_year'] != '2011']
freqs = Counter(df.zipper)
print(freqs)

mf.describe()

Counter({'9411': 467, '9460': 430, '9410': 319, '9461': 214, '9511': 213, '9412': 184, '9512': 171, '9470': 163, '9406': 150, '9454': 141, '9408': 110, '9450': 103, '9505': 99, '9480': 92, '9440': 91, '9413': 86, '9404': 84, '9457': 80, '9459': 76, '9452': 73, '9401': 68, '9490': 58, '9402': 45, '9456': 45, '9453': 40, '9455': 39, '9430': 33, '9500': 33, '9400': 32, '9494': 31, '9513': 25, '9451': 24, '9407': 21, '9403': 21, '9496': 19, '9462': 18, '9458': 16, '9471': 13, '9493': 11, '9503': 10, '9492': 9, '9501': 9, '9415': 1})


,Sale_Price,Cap_Rate,Bldg SF,Floor Area Ratio,Year Built,Number Of Units,Longitude,Latitude,sqft,cap,...,log_price,log_sqft,log_far,log_units,bin_city,bin_yrbuilt,year,sale_year_qrt,yearbuilt,bin_zip
count,2.518000e+03,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,...,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,5.437624e+06,5.134980,18280.353058,1.184863,1947.763304,21.148928,-122.225733,37.702543,18280.353058,4.686259,...,14.667453,9.053909,-0.113775,2.365398,49.592931,65.773630,2014.081811,14.338324,194.366164,20.008340
std,1.514919e+07,1.642576,47132.768021,1.001046,25.855862,47.317385,0.196290,0.210514,47132.768021,1.665054,...,1.054239,0.966212,0.759459,0.937238,19.508133,25.830219,1.507121,1.501520,2.596749,11.653398
min,1.400000e+05,0.090000,439.000000,0.004905,1875.000000,2.000000,-122.603481,37.243408,439.000000,0.000000,...,11.849398,6.084499,-5.317445,0.693147,0.000000,1.000000,2012.000000,12.100000,187.000000,0.000000
25%,1.150000e+06,4.082500,4355.250000,0.534284,1924.000000,6.000000,-122.415011,37.565683,4355.250000,4.000000,...,13.955273,8.379137,-0.626829,1.791759,47.000000,42.000000,2013.000000,13.200000,192.000000,9.000000
50%,1.947500e+06,4.950000,6728.000000,0.780022,1959.000000,8.000000,-122.252222,37.775391,6728.000000,4.000000,...,14.482056,8.814032,-0.248434,2.079442,59.000000,77.000000,2014.000000,14.300000,195.000000,20.000000
75%,4.000000e+06,5.940000,12966.000000,1.560856,1964.000000,16.000000,-122.082707,37.807909,12966.000000,5.000000,...,15.201805,9.470086,0.445234,2.772589,62.000000,82.000000,2015.000000,15.400000,196.000000,26.000000
max,2.450000e+08,22.020000,800025.000000,9.857635,2016.000000,637.000000,-121.691135,38.158380,800025.000000,22.000000,...,19.316769,13.592398,2.288246,6.456770,77.000000,134.000000,2017.000000,17.100000,201.000000,42.000000


In [9]:
# formula must have variables in string, no spaces!
est = smf.ols(formula = 'np.log(Sale_Price) ~ (cap + (np.log(units)/np.log(sqft)) + yearbuilt) * C(zipper)', data = mf)
results = est.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     np.log(Sale_Price)   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                     114.6
Date:                Wed, 24 May 2017   Prob (F-statistic):               0.00
Time:                        09:13:33   Log-Likelihood:                -649.75
No. Observations:                2518   AIC:                             1717.
Df Residuals:                    2309   BIC:                             2936.
Df Model:                         208                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------

In [11]:
#'np.log(Sale_Price) ~ cap * (np.log(units)/np.log(sqft)) * sale_year_qrt * yearbuilt * C(zipper)'
a = pd.DataFrame({'cap':[5.45],'units':[6], 'sqft':[5000],'yearbuilt':[195], 'zipper':['9410']})
ans = results.predict(a)
price = np.exp(ans).astype(int)
ppsf = price/a.sqft
print('Price: ')
print(price)
print('Price per SQFT: ')
print(ppsf.values)

Price: 
[1981275]
Price per SQFT: 
[ 396.255]
